In [1]:
#| default_exp training.transcription.train

In [33]:
#| export
from pathlib import Path
from torch import Tensor
import torch
import torch.nn as nn
from torch.nn import Transformer
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import DataLoader, Dataset
from torchtext.vocab import vocab, Vocab
from sklearn.model_selection import train_test_split
import math
from timeit import default_timer as timer
from tqdm import tqdm
import pandas as pd
from collections import OrderedDict, Counter

from llm_mito_scanner.data.download import load_config, \
    get_latest_assembly_path
from llm_mito_scanner.training.transcription.generation import \
    BOS_TOK, EOS_TOK, UNK_TOK, PAD_TOK


DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [2]:
#| hide
config = load_config()

In [3]:
#| hide
data_path = Path(config.get("data_path"))
data_raw_path = data_path / "raw"
assemblies_path = data_raw_path / "assemblies"
latest_assembly_path = get_latest_assembly_path(assemblies_path)
chromosomes_path = latest_assembly_path / "chromosomes"
training_data_path = latest_assembly_path / "training"
transcription_data_path = training_data_path / "transcription"
sequences_data_path = transcription_data_path / "sequences"

random_state = 42

In [4]:
#| hide
training_index = pd.read_csv(transcription_data_path / "training_sequence_idx.csv", nrows=10)
training_index.head()

,chromosome,geneid,transcriptid,start,end,type
0,NC_000019.10,GeneID:946,XM_011527538.4,57,121,intron-small
1,NC_000006.12,GeneID:124901227,XM_047419609.1,0,64,intron-small
2,NC_000002.12,GeneID:1768,XM_011532649.3,251297,251361,intron
3,NC_000010.11,GeneID:657,NM_001406576.1,86904,86968,intron
4,NC_000005.10,GeneID:3187,NM_001257293.2,2153,2217,intron


In [5]:
#| hide
# Get vocab
training_data = pd.read_csv(transcription_data_path / "training_data.csv", nrows=10)
training_data.head()

,input,target,position,chromosome,type
0,"['G', 'C', 'T', 'A', 'C', 'C', 'G', 'C', 'T', ...","['G', 'C', 'U', 'A', 'C', 'C', 'G', 'C', 'U', ...",57,NC_000019.10,intron-small
1,"['C', 'C', 'C', 'C', 'A', 'C', 'A', 'G', 'G', ...","['<intron>', '<intron>', '<intron>', '<intron>...",183,NC_000019.10,NaN
2,"['T', 'T', 'T', 'G', 'A', 'G', 'A', 'T', 'G', ...","['<intron>', '<intron>', '<intron>', '<intron>...",39751,NC_000019.10,NaN
3,"['A', 'T', 'G', 'T', 'C', 'T', 'G', 'G', 'A', ...","['<null>', '<null>', '<null>', '<null>', '<nul...",1458,NC_000019.10,NaN
4,"['T', 'T', 'A', 'A', 'G', 'C', 'A', 'G', 'C', ...","['<intron>', '<intron>', '<intron>', '<intron>...",20273,NC_000019.10,NaN


In [67]:
#| export
def list_string_to_list(l: str, sep=", ") -> list[str]:
    return l.replace("'", "").strip("][").split(sep)

In [239]:
#| export
def get_vocab(transcription_data_path: Path, special_tokens: list[str] = [UNK_TOK, BOS_TOK, EOS_TOK, PAD_TOK]) -> Vocab:
    vocab_path = transcription_data_path / "vocab.pt"
    if vocab_path.exists():
        token_vocab = torch.load(vocab_path)
        return token_vocab
    else:
        training_data_path = transcription_data_path / "training_data.csv"
        token_counter = Counter()
        for col in ['input', 'target']:
            sequences = pd.read_csv(training_data_path, usecols=[col])[col].apply(list_string_to_list).tolist()
            sequence_counters = list(map(Counter, sequences))
            token_counter = token_counter + sum(sequence_counters, Counter())
        token_vocab = vocab(token_counter, specials=special_tokens)
        token_vocab.set_default_index(token_vocab[UNK_TOK])
        torch.save(token_vocab, vocab_path)
        return token_vocab

In [240]:
#| hide
training_vocab = get_vocab(transcription_data_path)

In [242]:
#| hide
len(training_vocab)

12

In [243]:
#| hide
training_vocab["<intron>"]

10

In [136]:
#| export
UNK_IDX = None
PAD_IDX = None
BOS_IDX = None
EOS_IDX = None

def set_vocab_idx(vocab: Vocab):
    global UNK_IDX, PAD_IDX, BOS_IDX, EOS_IDX
    UNK_IDX = vocab[UNK_TOK]
    PAD_IDX = vocab[PAD_TOK]
    BOS_IDX = vocab[BOS_TOK]
    EOS_IDX = vocab[EOS_TOK]

In [137]:
#| hide
training_vocab[UNK_TOK]

0

In [138]:
#| hide
set_vocab_idx(training_vocab)

In [139]:
#| export
class TranscriptionDataset(Dataset):
    def __init__(self, path: Path, train: bool, **train_test_split_kwargs) -> None:
        self.path = path
        self.train = train
        data_train, data_test = train_test_split(pd.read_csv(path), **train_test_split_kwargs)
        self.data = data_train if train else data_test
    
    def __len__(self) -> int:
        return self.data.shape[0]

    def __getitem__(self, idx) -> tuple[str, str, int]:
        row = self.data.iloc[idx]
        input_str = list_string_to_list(row.input)
        target_str = list_string_to_list(row.target)
        position = row.position
        return input_str, target_str, position

In [140]:
#| hide
# Get train, test datasets
transcription_dataset_train = TranscriptionDataset(transcription_data_path / "training_data.csv", True)
transcription_dataset_test = TranscriptionDataset(transcription_data_path / "training_data.csv", False)

In [141]:
#| hide
example_input, example_target, example_position = transcription_dataset_train[0]
[len(x) if type(x) == list else x for x in transcription_dataset_train[0]]

[64, 64, 213016]

In [142]:
#| export
# helper Module that adds positional encoding to the token embedding to introduce a notion of word order.
class PositionalEncoding(nn.Module):
    def __init__(self,
                 emb_size: int,
                 dropout: float,
                 maxlen: int = 5000):
        super(PositionalEncoding, self).__init__()
        den = torch.exp(- torch.arange(0, emb_size, 2)* math.log(10000) / emb_size)
        pos = torch.arange(0, maxlen).reshape(maxlen, 1)
        pos_embedding = torch.zeros((maxlen, emb_size))
        pos_embedding[:, 0::2] = torch.sin(pos * den)
        pos_embedding[:, 1::2] = torch.cos(pos * den)
        pos_embedding = pos_embedding.unsqueeze(-2)

        self.dropout = nn.Dropout(dropout)
        self.register_buffer('pos_embedding', pos_embedding)

    def forward(self, token_embedding: Tensor):
        return self.dropout(token_embedding + self.pos_embedding[:token_embedding.size(0), :])

# helper Module to convert tensor of input indices into corresponding tensor of token embeddings
class TokenEmbedding(nn.Module):
    def __init__(self, vocab_size: int, emb_size):
        super(TokenEmbedding, self).__init__()
        self.embedding = nn.Embedding(vocab_size, emb_size)
        self.emb_size = emb_size

    def forward(self, tokens: Tensor):
        return self.embedding(tokens.long()) * math.sqrt(self.emb_size)

# Seq2Seq Network
class Seq2SeqTransformer(nn.Module):
    def __init__(self,
                 num_encoder_layers: int,
                 num_decoder_layers: int,
                 emb_size: int,
                 nhead: int,
                 src_vocab_size: int,
                 tgt_vocab_size: int,
                 dim_feedforward: int = 512,
                 dropout: float = 0.1):
        super(Seq2SeqTransformer, self).__init__()
        self.transformer = Transformer(d_model=emb_size,
                                       nhead=nhead,
                                       num_encoder_layers=num_encoder_layers,
                                       num_decoder_layers=num_decoder_layers,
                                       dim_feedforward=dim_feedforward,
                                       dropout=dropout)
        self.generator = nn.Linear(emb_size, tgt_vocab_size)
        self.src_tok_emb = TokenEmbedding(src_vocab_size, emb_size)
        self.tgt_tok_emb = TokenEmbedding(tgt_vocab_size, emb_size)
        self.positional_encoding = PositionalEncoding(
            emb_size, dropout=dropout)

    def forward(self,
                src: Tensor,
                trg: Tensor,
                src_mask: Tensor,
                tgt_mask: Tensor,
                src_padding_mask: Tensor,
                tgt_padding_mask: Tensor,
                memory_key_padding_mask: Tensor):
        src_emb = self.positional_encoding(self.src_tok_emb(src))
        tgt_emb = self.positional_encoding(self.tgt_tok_emb(trg))
        outs = self.transformer(src_emb, tgt_emb, src_mask, tgt_mask, None,
                                src_padding_mask, tgt_padding_mask, memory_key_padding_mask)
        return self.generator(outs)

    def encode(self, src: Tensor, src_mask: Tensor):
        return self.transformer.encoder(self.positional_encoding(
                            self.src_tok_emb(src)), src_mask)

    def decode(self, tgt: Tensor, memory: Tensor, tgt_mask: Tensor):
        return self.transformer.decoder(self.positional_encoding(
                          self.tgt_tok_emb(tgt)), memory,
                          tgt_mask)

In [143]:
#| hide
torch.manual_seed(random_state)

SRC_VOCAB_SIZE = TGT_VOCAB_SIZE = len(training_vocab)
EMB_SIZE = 128
NHEAD = 4
FFN_HID_DIM = 128
BATCH_SIZE = 64
NUM_ENCODER_LAYERS = 3
NUM_DECODER_LAYERS = 3

transformer = Seq2SeqTransformer(NUM_ENCODER_LAYERS, NUM_DECODER_LAYERS, EMB_SIZE,
                                 NHEAD, SRC_VOCAB_SIZE, TGT_VOCAB_SIZE, FFN_HID_DIM)

for p in transformer.parameters():
    if p.dim() > 1:
        nn.init.xavier_uniform_(p)

transformer = transformer.to(DEVICE)

loss_fn = torch.nn.CrossEntropyLoss(ignore_index=PAD_IDX)

optimizer = torch.optim.Adam(transformer.parameters(), lr=0.0001, betas=(0.9, 0.98), eps=1e-9)

In [144]:
#| export
# helper function to club together sequential operations
def sequential_transforms(*transforms):
    def func(txt_input):
        for transform in transforms:
            txt_input = transform(txt_input)
        return txt_input
    return func

# function to add BOS/EOS and create tensor for input sequence indices
def tensor_transform(token_ids: list[int]):
    return torch.cat((torch.tensor([BOS_IDX]),
                      torch.tensor(token_ids),
                      torch.tensor([EOS_IDX])))

In [145]:
#| export
text_transform = None
def set_text_transform(
        text_vocab: Vocab
):
    global text_transform
    text_transform = sequential_transforms(
    text_vocab, # Numericalization
    tensor_transform)

In [146]:
#| hide
set_text_transform(training_vocab)

In [147]:
#| hide
example_input_transformed = text_transform(example_input)
example_target_transformed = text_transform(example_target)
training_vocab.lookup_token(10)

'<intron>'

In [250]:
#| export
# function to collate data samples into batch tensors
def collate_fn(batch: list[tuple[str, str, int]]):
    global PAD_IDX
    src_batch, tgt_batch = [], []
    for src_sample, tgt_sample, pos in batch:
        src_batch.append(text_transform(src_sample))
        tgt_batch.append(text_transform(tgt_sample))

    src_batch = pad_sequence(src_batch, padding_value=PAD_IDX)
    tgt_batch = pad_sequence(tgt_batch, padding_value=PAD_IDX)
    return src_batch, tgt_batch

In [157]:
#| hide
example_loader = DataLoader(transcription_dataset_train, batch_size=32, collate_fn=collate_fn)
example_batch = next(iter(example_loader))

In [158]:
#| hide
len(example_batch)

2

In [159]:
#| hide
example_batch[0].shape

torch.Size([66, 32])

In [160]:
#| hide
training_vocab.lookup_token(0)

'<unk>'

In [166]:
#| hide
try:
    del model
except:
    pass
torch.cuda.empty_cache()
model = transformer
model.train()
losses = 0

In [167]:
#| hide
batch_loss = None

In [168]:
#| hide
chunks_attached = False

In [169]:
#| hide
if not chunks_attached:
    example_input = example_batch[0].to(DEVICE)
    example_target = example_batch[1].to(DEVICE)
    chunks_attached = True

In [178]:
#| hide
example_target_input = example_target[:-1, :]

In [190]:
#| hide
example_target_out = example_target[1:, :]

In [191]:
#| hide
example_input.shape, example_target.shape, example_target_input.shape, example_target_out.shape

(torch.Size([66, 32]),
 torch.Size([66, 32]),
 torch.Size([65, 32]),
 torch.Size([65, 32]))

In [173]:
#| export
def generate_square_subsequent_mask(sz):
    mask = (torch.triu(torch.ones((sz, sz), device=DEVICE)) == 1).transpose(0, 1)
    mask = mask.float().masked_fill(mask == 0, float('-inf')).masked_fill(mask == 1, float(0.0))
    return mask


def create_mask(src, tgt):
    src_seq_len = src.shape[0]
    tgt_seq_len = tgt.shape[0]

    tgt_mask = generate_square_subsequent_mask(tgt_seq_len)
    src_mask = torch.zeros((src_seq_len, src_seq_len), device=DEVICE).type(torch.bool)

    src_padding_mask = (src == PAD_IDX).transpose(0, 1)
    tgt_padding_mask = (tgt == PAD_IDX).transpose(0, 1)
    return src_mask, tgt_mask, src_padding_mask, tgt_padding_mask

In [195]:
#| hide
masks_made = False

In [196]:
#| hide
# del example_src_mask, example_tgt_mask, example_src_padding_mask, example_tgt_padding_mask

In [197]:
#| hide
if not masks_made:
    example_src_mask, example_tgt_mask, example_src_padding_mask, example_tgt_padding_mask = create_mask(
        example_input, 
        example_target_input
        )
    masks_made = True

for t in [example_src_mask, example_tgt_mask, example_src_padding_mask, example_tgt_padding_mask]:
    print(t.shape)

torch.Size([66, 66])
torch.Size([65, 65])
torch.Size([32, 66])
torch.Size([32, 65])


In [198]:
#| hide
logits = model(
    example_input, 
    example_target_input, 
    example_src_mask, 
    example_tgt_mask,
    example_src_padding_mask, 
    example_tgt_padding_mask, 
    example_src_padding_mask
    )

In [199]:
#| hide
optimizer.zero_grad()

loss = loss_fn(logits.reshape(-1, logits.shape[-1]), example_target_out.reshape(-1))
# loss.backward()
# optimizer.step()
# losses += loss.item()

In [201]:
#| hide
loss

tensor(3.0219, device='cuda:0', grad_fn=<NllLossBackward0>)

In [207]:
#| hide
torch.cuda.empty_cache()
try:
    for var in [
        model,
        optimizer,
        loss,
        batch_loss,
        example_input,
        example_target,
        example_target_input,
        example_target_out,
        example_src_mask, 
        example_tgt_mask,
        example_src_padding_mask, 
        example_tgt_padding_mask,
        chunk_loss
    ]:
        try:
            del var
        except Exception:
            pass
except NameError as e:
    raise e

In [211]:
#| hide
len(example_loader), len(transcription_dataset_train) // 32

(844, 843)

In [245]:
#| export
def train_epoch(
        model, optimizer, loss_fn, 
        dataset: TranscriptionDataset,
        batch_size: int = 32,
        pbar_position: int = 1,
        limit: int = None):
    global DEVICE
    model.train()
    losses = 0
    train_dataloader = DataLoader(dataset, batch_size=batch_size, collate_fn=collate_fn)
    batches = len(train_dataloader)
    batch_pbar = tqdm(total=batches, position=pbar_position, leave=False, ncols=80, desc="Training")
    for i, (src, tgt) in enumerate(train_dataloader):
        src = src.to(DEVICE)
        tgt = tgt.to(DEVICE)

        tgt_input = tgt[:-1, :]

        src_mask, tgt_mask, src_padding_mask, tgt_padding_mask = create_mask(src, tgt_input)

        logits = model(src, tgt_input, src_mask, tgt_mask, src_padding_mask, tgt_padding_mask, src_padding_mask)

        optimizer.zero_grad()

        tgt_out = tgt[1:, :]
        loss = loss_fn(logits.reshape(-1, logits.shape[-1]), tgt_out.reshape(-1))
        loss.backward()

        optimizer.step()
        losses += loss.item()
        batch_pbar.update(1)
        if isinstance(limit, int):
            if i >= limit:
                batches = limit
                break

    batch_pbar.close()

    return losses / batches


def evaluate(
        model, loss_fn, 
        dataset: TranscriptionDataset,
        batch_size: int = 32,
        pbar_position: int = 1,
        limit: int = None):
    model.eval()
    losses = 0

    val_dataloader = DataLoader(dataset, batch_size=batch_size, collate_fn=collate_fn)
    batches = len(val_dataloader)
    eval_pbar = tqdm(total=batches, position=pbar_position, leave=False, ncols=80, desc="Evaluating")
    for i, (src, tgt) in enumerate(val_dataloader):
        # Break these up into reasonable sizes for the GPU
        src = src.to(DEVICE)
        tgt = tgt.to(DEVICE)

        tgt_input = tgt[:-1, :]

        src_mask, tgt_mask, src_padding_mask, tgt_padding_mask = create_mask(src, tgt_input)

        logits = model(src, tgt_input, src_mask, tgt_mask,src_padding_mask, tgt_padding_mask, src_padding_mask)

        tgt_out = tgt[1:, :]
        loss = loss_fn(logits.reshape(-1, logits.shape[-1]), tgt_out.reshape(-1))
        losses += loss.item()
        
        eval_pbar.update(1)
        if isinstance(limit, int):
            if i >= limit:
                batches = limit
                break
        
    eval_pbar.close()
    return losses / batches

In [246]:
#| hide
transformer = Seq2SeqTransformer(NUM_ENCODER_LAYERS, NUM_DECODER_LAYERS, EMB_SIZE,
                                 NHEAD, SRC_VOCAB_SIZE, TGT_VOCAB_SIZE, FFN_HID_DIM)

for p in transformer.parameters():
    if p.dim() > 1:
        nn.init.xavier_uniform_(p)

transformer = transformer.to(DEVICE)

loss_fn = torch.nn.CrossEntropyLoss(ignore_index=PAD_IDX)

optimizer = torch.optim.Adam(transformer.parameters(), lr=0.0001, betas=(0.9, 0.98), eps=1e-9)

example_train_loss = train_epoch(
    transformer, optimizer, loss_fn,
    transcription_dataset_train,
    batch_size = 32,
    pbar_position = 0,
    limit=4)

example_eval_loss = evaluate(
    transformer, loss_fn, 
    transcription_dataset_test,
    batch_size = 32,
    pbar_position = 0,
    limit=4)

example_train_loss, example_eval_loss

(2.904506206512451, 2.2545067965984344)

In [247]:
#| export
# function to generate output sequence using greedy algorithm
def greedy_decode(model, src, src_mask, max_len, start_symbol):
    src = src.to(DEVICE)
    src_mask = src_mask.to(DEVICE)

    memory = model.encode(src, src_mask)
    ys = torch.ones(1, 1).fill_(start_symbol).type(torch.long).to(DEVICE)
    for i in range(max_len-1):
        memory = memory.to(DEVICE)
        tgt_mask = (generate_square_subsequent_mask(ys.size(0))
                    .type(torch.bool)).to(DEVICE)
        out = model.decode(ys, memory, tgt_mask)
        out = out.transpose(0, 1)
        prob = model.generator(out[:, -1])
        _, next_word = torch.max(prob, dim=1)
        next_word = next_word.item()

        ys = torch.cat([ys,
                        torch.ones(1, 1).type_as(src.data).fill_(next_word)], dim=0)
        if next_word == EOS_IDX:
            break
    return ys


# actual function to translate input sentence into target language
def translate(model: torch.nn.Module, vocab: Vocab, src_sentence: str):
    model.eval()
    src = text_transform(src_sentence).view(-1, 1)
    num_tokens = src.shape[0]
    src_mask = (torch.zeros(num_tokens, num_tokens)).type(torch.bool)
    tgt_tokens = greedy_decode(
        model,  src, src_mask, max_len=num_tokens + 5, start_symbol=BOS_IDX).flatten()
    return " ".join(vocab.lookup_tokens(list(tgt_tokens.cpu().numpy()))).replace("<bos>", "").replace("<eos>", "")

In [248]:
#| hide
example_sequence = list("AGTGCCGTAA")
example_translation = translate(transformer, training_vocab, example_sequence)
example_translation

' <intron> <intron> <intron> <intron> <intron> <intron> <intron> <intron> <intron> <intron> <intron> <intron> <intron> <intron> <intron> <intron>'

In [251]:
#| hide
import nbdev; nbdev.nbdev_export()